# 정리

이 노트북에서는 일련의 이전 노트북에서 만든 모든 리소스를 정리하는 방법을 보여줍니다.

In [ ]:
import boto3
import json

In [ ]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

In [ ]:
%store -r

각 단계가 완전히 완료되고 나야 다음 단계가 작동한다는 점을 유의하세요. 오류가 발생하면 1~2분 정도 기다렸다가 다시 시도하세요.

## 캠페인 정리

맨 먼저 캠페인을 정리합니다. 캠페인의 기반이 되는 솔루션 버전을 삭제하려면 먼저 캠페인을 삭제해야 합니다.

아래 코드는 사용자가 만든 캠페인을 삭제합니다.

In [ ]:
personalize.delete_campaign(
    campaignArn = userpersonalization_campaign_arn
)
personalize.delete_campaign(
    campaignArn = sims_campaign_arn
)
personalize.delete_campaign(
    campaignArn = rerank_campaign_arn
)

## 이벤트 트래커 정리

다음으로 이벤트 트래커를 모두 정리합니다. 아래의 코드는 이벤트 트래커를 삭제합니다.

In [ ]:
personalize.delete_event_tracker(
    eventTrackerArn = event_tracker_arn
)

## 필터 정리

다음으로 필터를 모두 정리합니다. 아래의 코드는 사용자가 만든 모든 필터에 대해 작업을 반복하며 필터를 삭제합니다. 캠페인에 대한 오류가 발생하면 2분 동안 기다렸다가 아래 셀을 다시 실행해 보세요.

참고: 캠페인 삭제가 완료될 때까지 필터를 삭제할 수 없습니다. 캠페인을 삭제하는 데 몇 분 정도 걸릴 수 있습니다.

In [ ]:
for filter_arn in meta_filter_arns:
    personalize.delete_filter(
        filterArn = filter_arn
    )
for filter_arn in interaction_filter_arns:
    personalize.delete_filter(
        filterArn = filter_arn
    )
for filter_arn in decade_filter_arns:
    personalize.delete_filter(
        filterArn = filter_arn
    )

## 솔루션 정리

다음으로 솔루션을 정리합니다. 아래의 코드는 생성된 솔루션을 삭제합니다.

In [ ]:
personalize.delete_solution(
    solutionArn = user_personalization_solution_arn
)
personalize.delete_solution(
    solutionArn = sims_solution_arn
)
personalize.delete_solution(
    solutionArn = rerank_solution_arn
)

## 데이터 세트 정리

다음으로 데이터 세트를 정리합니다. 아래의 코드는 항목 및 상호 작용 데이터 세트를 삭제합니다.

In [ ]:
personalize.delete_dataset(
    datasetArn = items_dataset_arn
)
personalize.delete_dataset(
    datasetArn = interactions_dataset_arn
)

## 스키마 정리

다음으로 스키마를 정리합니다. 상호 작용 및 항목 스키마를 만들었으므로, 해당 스키마를 삭제합니다.

In [ ]:
personalize.delete_schema(
    schemaArn = interaction_schema_arn
)
personalize.delete_schema(
    schemaArn = itemmetadataschema_arn
)

## 데이터 세트 그룹 정리

마지막으로, 데이터 세트 그룹을 정리합니다.

In [ ]:
personalize.delete_dataset_group(
    datasetGroupArn = dataset_group_arn
)

## S3 버킷 및 IAM 역할 정리

먼저 역할을 삭제한 다음, 버킷을 비운 후 버킷을 삭제합니다.

In [ ]:
iam = boto3.client('iam')

삭제할 역할의 이름을 식별합니다.

아직 정책이 연결되어 있는 IAM 역할은 삭제할 수 없습니다. 따라서 관련 역할을 식별한 후, 해당 역할에 연결된 정책을 나열해 봅니다.

In [ ]:
iam.list_attached_role_policies(
    RoleName = role_name
)

아래 코드를 사용하여 위의 결과에서 정책을 분리해야 합니다. 연결된 정책마다 이 작업을 반복합니다.

In [ ]:
iam.detach_role_policy(
    RoleName = role_name,
    PolicyArn = "arn:aws:iam::aws:policy/service-role/AmazonPersonalizeFullAccess"
)

In [ ]:
iam.detach_role_policy(
    RoleName = role_name,
    PolicyArn = 'arn:aws:iam::aws:policy/AmazonS3FullAccess'
)

이제 IAM 역할을 삭제할 수 있습니다.

In [ ]:
iam.delete_role(
    RoleName = role_name
)

S3 버킷을 삭제하려면 먼저 버킷이 비어 있어야 합니다. S3 버킷을 삭제하는 가장 쉬운 방법은 AWS 콘솔에서 S3으로 이동하여 버킷의 객체를 삭제한 다음 S3 버킷 자체를 삭제하는 것입니다.